In [1]:
import math
import pandas as pd
import tbapy
from lxml import etree
from frccolors import RoboColor

from haversine import haversine
tba = tbapy.TBA('LenYI5DMTy1bQIoP4ralqFVd5g5JvTY9YBEBAubPktLTelEdPukJZ1RVLIV1Ypfu')
not_found_string = "N/A"
frc_county_loc = "/home/will/Documents/Projects/frc_counties/data/frc_teams_counties.csv"
county_loc = "/home/will/Documents/Projects/frc_counties/data/counties_loc.csv"
frc_beaten_loc = "/home/will/Documents/Projects/frc_counties/data/beaten_teams.csv"
imperialism_loc = "/home/will/Documents/Projects/frc_counties/data/imperialism_map_start.csv"
target_loc = "frc_undefeated_map.svg"
svg_loc = "/home/will/Documents/Projects/frc_counties/data/counties.svg"
custom_colors_loc = "/home/will/Documents/Projects/frc_counties/data/team_colors.csv"
roboColor = RoboColor(custom_colors_loc)

min_color_val = 25

In [2]:
# library funcs for modifying svg file
def writeNewSVGWithColorDictionary(color_dict, tooltips_dict = None):
    tree = etree.parse(svg_loc)
    root = tree.getroot()
    paths = root[2]
    for path in paths:
        path_county_id = path.attrib["id"]
        if path_county_id not in color_dict:
            continue
        path.attrib["style"] = "fill: " + color_dict[path_county_id] + ";"
        if tooltips_dict != None and path_county_id in tooltips_dict:
            path[0].text += " - " + str(tooltips_dict[path_county_id])
    etree.ElementTree(root).write(target_loc, pretty_print=True)

def getFipsString(fips):
    fips = str(fips)
    while len(fips) < 5:
        fips = "0" + fips
    return "c" + fips

In [3]:
# pull in FRC county data to give default values
frczips = pd.read_csv(frc_county_loc)
# remove all beaten teams from this list - TODO
frcbeaten = pd.read_csv(frc_beaten_loc)
indices_to_remove = []
for index, row in frczips.iterrows():
    if row["Team Number"] in frcbeaten["Team"].values:
        indices_to_remove.append(index)
frczips = frczips.drop(indices_to_remove)
frczips.head()

,Team Number,Country,State,City,Postal Code,State Code,fips
426,1323,USA,California,Madera,93637,CA,"c06019,c06039"


In [4]:
# pull in county location data

def distBetweenTwoCounties(rowdata1, rowdata2): #pass in rows of data from countylocs to get distance between the two
    return haversine((rowdata1["Latitude"], rowdata1["Longitude"]), (rowdata2["Latitude"], rowdata2["Longitude"]))

countylocs = pd.read_csv(county_loc)
allcounties = countylocs
#countylocs = countylocs.head(100) #uncomment to subset
def getRowDataFromFIPS(fips):
    og_fips = int(fips[1:])
    return allcounties[allcounties["FIPS"] == og_fips]


countylocs.head()



,Sort [1],State,FIPS,County [2],Population(2010),Land Areakm²,Land Areami²,Water Areakm²,Water Areami²,Total Areakm²,Total Areami²,Latitude,Longitude
0,1,AL,1001,Autauga,"54,571","1,539.58",594.436,25.776,9.952,"1,565.36",604.388,32.536382,-86.644490
1,2,AL,1003,Baldwin,"182,265","4,117.52","1,589.78","1,133.19",437.527,"5,250.71","2,027.31",30.659218,-87.746067
2,3,AL,1005,Barbour,"27,457","2,291.82",884.876,50.865,19.639,"2,342.68",904.515,31.870670,-85.405456
3,4,AL,1007,Bibb,"22,915","1,612.48",622.582,9.289,3.587,"1,621.77",626.169,33.015893,-87.127148
4,5,AL,1009,Blount,"57,322","1,669.96",644.776,15.157,5.852,"1,685.12",650.628,33.977448,-86.567246


In [5]:
# create datastructure for county assignment, and give default values
county_owners = {}
for index, row in frczips.iterrows():
    county_fips = row["fips"].split(",")
    team_num = row["Team Number"]
    for county in county_fips:
        if county not in county_owners:
            county_owners[county] = [team_num]
        else:
            county_owners[county].append(team_num)
# print teams that were loaded
for county in county_owners:
    print("Found undefeated team: " + str(county_owners[county]))

Found undefeated team: [1323]
Found undefeated team: [1323]


In [6]:
# generate list of all counties fips
all_fips = []
for index, row in countylocs.iterrows():
    all_fips.append(getFipsString(row["FIPS"]))
# fill in blank counties with closest
county_assignment = {} #structure: county_ass[county] = (lowDist, [teams])
# add teams home county first w/ dist 0
for index, row in frczips.iterrows():
    og_fips = row["fips"].split(",")
    for fip in og_fips:
        if fip in county_assignment:
            county_assignment[fip][1].append(row["Team Number"])
        else:
            county_assignment[fip] = (0, [row["Team Number"]])
            
# for each home county, loop through all non-home counties and check if it's empty, if so assign dist and claim
prog = 1
memoization_helper = {}
for home_fips in county_owners:
    home_teams = county_assignment[home_fips][1]
    for index, row in countylocs.iterrows():
        away_fips = getFipsString(row["FIPS"])
        memo_keys = [home_fips, away_fips]
        memo_keys.sort()
        memo_key = "".join(memo_keys)
        if away_fips in county_assignment:
            if county_assignment[away_fips][0] > 0:
                dist = -1
                if memo_key in memoization_helper:
                    dist = memoization_helper[memo_key]
                else:
                    dist = distBetweenTwoCounties(row, getRowDataFromFIPS(home_fips))
                    memoization_helper[memo_key] = dist
                if dist < county_assignment[away_fips][0]:
                    county_assignment[away_fips] = (dist, home_teams)
        else: # add to
            dist = distBetweenTwoCounties(row, getRowDataFromFIPS(home_fips))
            county_assignment[away_fips] = (dist, home_teams)
    if prog < 6 or prog % 25 == 0:
        print("Finished with home county " + str(prog) + "/" + str(len(county_owners)))
    prog += 1

Finished with home county 1/2
Finished with home county 2/2


In [7]:
print(len(county_assignment))

3144


In [8]:
# create color dictionary for displaying data
color_dict = {}
hover_dict = {}
for county_fips_key in county_assignment:
    if len(county_assignment[county_fips_key][1]) > 1:
        color_dict[county_fips_key] = roboColor.getTeamRGB(county_assignment[county_fips_key][1]) #getTeamRGB(county_assignment[county_fips_key][1])#genCSSRGB(0, 0, 0)
        toAdd = str(county_assignment[county_fips_key][1][0])
        for i in range(1, len(county_assignment[county_fips_key][1])):
            toAdd += ", " + str(county_assignment[county_fips_key][1][i])
        hover_dict[county_fips_key] = toAdd
    else:
        color_dict[county_fips_key] = roboColor.getTeamRGB(county_assignment[county_fips_key][1][0])
        hover_dict[county_fips_key] = str(county_assignment[county_fips_key][1][0])

In [9]:
# write to file
writeNewSVGWithColorDictionary(color_dict, hover_dict)

In [10]:
# write county_assignment to file
#writer = open(imperialism_loc, "w")
#writer.write("County,Teams\n")
#for county in county_assignment:
#    toWrite = county + ",\""
#    for team in county_assignment[county][1]:
#        toWrite += str(team) + ","
#    writer.write(toWrite[:len(toWrite) - 1] + "\"\n")
#writer.close()